![R-Kenntnis](../Pics/header.png "R-Kenntnis")

<div class="alert alert-block alert-info">

* **Titel:** R-Kenntnis Lösung Übungen 08: Zeitreihenanalyse mit R
* **Autor:** Prof. Dr. Denis Royer
* **Datum:** 04.12.2021 (Version 1.0)

</div>

# BI - R-Kenntnis Lösung Übungen 08: Zeitreihenanalyse mit R

<div class="alert alert-block alert-warning">
<b>Wichtig:</b> Bevor wir loslegen, müssen wir zunächst einmal ein paar vorbereitende Dinge erledigen:

* Die notwendigen Packages laden (bspw. `tidyverse`)
* Datensatz laden (siehe *Data/marketing.csv*)

</div>

In [ ]:
# Laden der notwendigen Libraries
library(tidyverse)
library(ggplot2)
library(forecast)
library(lubridate)

# Übungsaufgaben: Analyse von Passagierdaten

## Datensatz und Aufgabe
Der Datensatz `AirPassengers` enthält die Daten für eine Fluggesellschaft und deren monatliches Passagieraufkommen für den Zeitraum von 1949 - 1960.

In [ ]:
data("AirPassengers")
# Ausgabe der Daten in einer druckfähigen Tabelle 
# mittels pander()
AirPassengers

![Übungsaufgabe](../Pics/excercise.png "Übungsaufgabe")
***Folgende Aufgaben:***


* Analysieren Sie die Daten und erstellen Sie eine Zeitreihenanalyse.
* Leiten Sie ein adäquates ARIMA Modell ab.
* Erstellen Sie mit Ihrem Modell eine Prognose für die Jahre 1961 - 1965.

Siehe auch folgende Links:

* <http://rstudio-pubs-static.s3.amazonaws.com/311446_08b00d63cc794e158b1f4763eb70d43a.html>
* <https://www.datascience.com/blog/introduction-to-forecasting-with-arima-in-r-learn-data-science-tutorials>
* <https://blog.exxcellent.de/datenanalyse-mit-r-so-erstellen-sie-interaktive-prognoseplots>

## Laden der Daten

In [ ]:
# Laden des Datensatzes
AirPassengers.ts <- AirPassengers
autoplot(AirPassengers.ts)

## Differencing: Identifikation von Trend- und Saison-Aspekten
Im ersten Schritt geht es nun darum, wie viele Differencing Schritte notwendig sind.

Wir wollen eine Zeitreihe, die nach dem Differencing keine Trend- oder Saison-Aspekte enthält.

In [ ]:
# Trend Differencing --> d=1
AirPassengers.ts.diff_trend <-diff(AirPassengers.ts)
# Saison Differencing d=0/D=1
AirPassengers.ts.diff_saison <- diff(AirPassengers.ts, lag = 12)
# Trend und Saison Differencing kombiniert--> d=1 / D=1
AirPassengers.ts.diff_trendsaison <- diff(diff(AirPassengers.ts, lag = 12))

In [ ]:
par(mfrow = c(2, 2))
# Plot der Zeitreihe
plot(AirPassengers.ts)
# Plot mit Differencing des Trends
plot(AirPassengers.ts.diff_trend)
# Plot mit Differencing der Saison (12)
plot(AirPassengers.ts.diff_saison)
# Kombiniertes Differencing (Trend & Saison)
plot(AirPassengers.ts.diff_trendsaison)

In [ ]:
# mit ggtsdisplay()
ggtsdisplay(AirPassengers.ts)
ggtsdisplay(AirPassengers.ts.diff_trend)
ggtsdisplay(AirPassengers.ts.diff_saison)
ggtsdisplay(AirPassengers.ts.diff_trendsaison)

In [ ]:
AirPassengers.ts.stl <- stl(AirPassengers.ts, "per")
plot(AirPassengers.ts.stl)

## Konfiguration des Modells

In [ ]:
AirPassengers.fit.auto = auto.arima(AirPassengers.ts, 
                                    trace = TRUE, 
                                    seasonal = TRUE, 
                                    approximation = FALSE, 
                                    ic = "aicc", 
                                    stepwise=FALSE)

summary(AirPassengers.fit.auto)

In [ ]:
AirPassengers.fit.auto %>% 
  residuals() %>% 
  ggtsdisplay()

## Überprüfung des Modells

In [ ]:
checkresiduals(AirPassengers.fit.auto)

In [ ]:
AirPassengers.yr_forecast2 <- forecast(AirPassengers.fit.auto, h = 10*12)
# AirPassengers.yr_forecast2
plot(AirPassengers.yr_forecast2)
autoplot(AirPassengers.fit.auto)

## Vergleich des Modells und der Vorhersage mittels Testdaten

In [ ]:
# Check des Modells
hold <- window(ts(AirPassengers.ts), start=97)

fit_no_holdout = arima(ts(AirPassengers.ts[-c(97:144)]), order=c(2,1,1), seasonal = c(0,1,0))

fcast_no_holdout <- forecast(fit_no_holdout,h=48)
plot(fcast_no_holdout, main=" ", col="green",lwd=3)
lines(ts(AirPassengers.ts),col="red")